In [ ]:

import importlib

if not importlib.util.find_spec("my_first_ltr"): # type: ignore
    %pip install -qqq git+https://github.com/algolia/my-first-learning-to-rank

# What's Learning To Rank ?

As the name indicates, it's training to rank stuff 

## Why rank ? And what's ranking here ?

## What's ranking model ?

it's a function that maps an item to it's relevance score.

# It all starts with data

- a user search history
- a subset of imdb dataset

We did all the nasty pre-processing and cleaning for you - so you can just have fun! 


At home, you can try with your search history if you'd like!
# FIXME: if so time, clean up the data prep script in a notebook

## What's our score here ?

- What's your idea ?

- We consider that if the user watched the movie, it was highly relevant
- We consider that if the user added it to it's watchlist, it was relevant, but not the right mood at that time
- We consider that if the user clicked on a movie, it showed some interest but it wasn't that relevant

In [ ]:
from my_first_ltr.utils import load_dataset

# FIXME: check if once repo is public that loading works
dataset = load_dataset(local=True)
dataset.head(5)



# FIXME: stuff we've learn and you should not waist time on
# - negative samples


# FIXME: show a specific query with results examples ?

# What are our features

# Then comes a model

PointWise
PairWise
ListWise

In [ ]:
from my_first_ltr.train_utils import build_pool, dataset_split


train_df, test_df, val_df = dataset_split(dataset)

train_pool = build_pool(train_df, "train")
test_pool = build_pool(test_df, "test")
val_pool = build_pool(val_df, "validation")




## RMSE pointwise

In [ ]:

from catboost import CatBoostRanker

model = CatBoostRanker(loss_function="RMSE", depth=6, learning_rate=0.15, thread_count=1, iterations=500, random_seed=0)
                    #    custom_metric=["NDCG:top=-1;type=Base;denominator=LogPosition;hints=skip_train~false"]
#                        )

# model = CatBoost({"loss_function":"YetiRank", "depth":6, "learning_rate":0.15, "thread_count":1, "iterations":500, "random_seed":0, "verbose":False})
#                        custom_metric=["NDCG:top=2;type=Base;denominator=LogPosition;hints=skip_train~false"])

# model = CatBoostRegressor(iterations=10000, depth=6, learning_rate=0.15, loss_function='RMSE', verbose=50)

model.fit(train_pool, eval_set=test_pool, plot=True, metric_period=1)



In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

val_df["pred_score"] = model.predict(val_pool)


results = val_df[["score", "pred_score"]]

print("Predictions vs Actuals:")
print(results.head())


rmse = np.sqrt(mean_squared_error(results["score"], results["pred_score"]))
mae = mean_absolute_error(results["score"], results["pred_score"])

print("\nEvaluation Metrics:")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")


## Feature importance

In [ ]:
import pandas as pd
from catboost import CatBoost, Pool
from my_first_ltr.train_utils import keep_input_features
import matplotlib.pyplot as plt

def cross_dataset_shape_cascade( m: CatBoost, X: pd.DataFrame, pools: dict[str, Pool]) -> None:
    """Shap values average per feature accross all dataset."""
    df_feature_importance = pd.DataFrame(
        data={k: m.get_feature_importance(pool) for k, pool in pools.items()}, index=X.columns
    )

    plt.close("all")
    df_feature_importance.plot.barh(figsize=(10, 12))
    plt.title("cross dataset - importance of each feature")
    plt.legend()
    plt.show()

    # plt.savefig(os.path.join(path, f"features_importances_{config.impl_name}.png"), format="png", bbox_inches="tight")

cross_dataset_shape_cascade(model, keep_input_features(train_df), {"train": train_pool, "test": test_pool, "val": val_pool})